In [2]:
from config import VK_TOKEN

In [3]:
import vk_api

vk_session = vk_api.VkApi(token=VK_TOKEN, app_id=5820017)
vk_session._auth_token()

vk = vk_session.get_api()

In [4]:
tools = vk_api.VkTools(vk_session)

In [5]:
vegan_recipes = tools.get_all_iter("wall.search", 20, 
                                   values={"owner_id": -35838585, "query": "#vegan", "owners_only": True})

In [6]:
import tqdm
import json

In [7]:
# downloaded_vegan_recipes = []
# for recipe in tqdm.tqdm(vegan_recipes):
#     downloaded_vegan_recipes.append(recipe)
# json.dump(downloaded_vegan_recipes, open("assets/downloaded_vegan_recipes.json", "w"))
downloaded_vegan_recipes = json.load(open("assets/downloaded_vegan_recipes.json"))

In [8]:
import re
from fuzzywuzzy import process

def process_recipe(recipe):
    tokens = recipe['text'].split()
    # best occurence of fuzzed ингредиент word
    ingr_word = process.extractOne("ингредиент", tokens)[0]
    ingr_idx = recipe['text'].index(ingr_word)
    # best occurence of fuzzed приготовление word
    prigot_word = process.extractOne("приготовление", tokens)[0]
    prigot_idx = recipe['text'].index(prigot_word)
    
    name = recipe['text'][:ingr_idx].strip()
    name, *description = name.rsplit("\n")
    description = "\n".join(description).strip()
    ingredients = recipe['text'][ingr_idx+len(ingr_word):prigot_idx].split("\n")
    ingredients = [it.strip() for it in ingredients if it.strip()]
    script = recipe['text'][prigot_idx+len(prigot_word):].split("\n")
    script = [it.strip() for it in script if it.strip()]
    return name, description, ingredients, script

In [9]:
for recipe in downloaded_vegan_recipes[5:10]:
    name, description, ingredients, script = process_recipe(recipe)
    print("название", name)
    print("описание", description)
    print("игредиенты", ingredients)
    print("приготовление", script)

название Салат с цветной капустой и горчичной заправкой
описание 
игредиенты ['(на 4-6 порций):', 'Для салата:', '1 кочан цветной капусты, нарезанный на соцветия', '1 ст. нарезанного огурца', '1 ст. нарезанных помидоров', '1\\4 ст. нарезанного зелёного лука', 'Поместите все ингредиенты в миску, перемешайте.', 'Для заправки:', '1\\2 ст. кешью, предварительно замоченных в воде', '1\\2 ст. воды', '1 размоченный финик', '1 ч.л. сухого горчичного порошка', '2 ст.л. кокосового масла', '1-2 ст.л. укропа', 'сок и цедра 1\\2 лимона', '2 измельчённых зубчика чеснока', 'соль и перец по вкусу', '1\\2 ст. нарезанных грецких орехов']
приготовление ['Поместите все ингредиенты, кроме укропа, чеснока, соли и перца в блендер. Взбейте до однородной консистенции. Добавьте оставшиеся цедру, укроп, чеснок, соль и перец. Взбейте ещё несколько раз. Вылейте поверх овощей, присыпьте орешками.', 'Приятного аппетита!', '#салаты@vegarecipes #веган@vegarecipes #vegan@vegarecipes #ПП@vegarecipes']
название Омлет из 

In [10]:
# ... \d штук|ст.|стакан|зубчика
# ... \d столовая ложка|по вкусу
# \d (финик|размоченный финик)
# (\d\\\d|\d\\\\d|\d,\d|\d-\d) ст.
# (¼|½)
# ... \d г

# если ничего, то нормализовать

In [243]:
different_measures = ["щеп(отка)?","ч\.\s?л","((ч(айн)?|ст)(олов)?\.?(ые|ых|ая|ой)?)?\s?ложе?к(а|и)?",
                      "гр?(амм)?", "ст\.\s?л", "зуб(чик)?\.?(а|ов)?","шт(ук(а|и)?)?", "веточк(а|и)",
                      "кусоч\.?(ка|ек)?", "кружк(а|и)?", "ломтик", "стеб(л(я|ей|евой)|ель(ка)?|)",
                      "чашк(а|и)?", "банк(а|и)?", "пуч(ок)?","килограмм","по( |\-)вкусу",
                      "лист(ы|ья|а|ьев|ов|ик(а|ов)?|очк(а|ов|и)?)", "с?м", "струч(ок|к(а|ов))",
                      "коч(ан)?","кг", "м?л","головк(а|и)","ст(ак)?\.?(ан|ана)?"]  # group3
digit_pattern = "\d+((,|\.)?\d*)?"
number_pattern = f"(\d*\s*(½|¼|⅔|⅛|⅓)|{digit_pattern}(\/|\\|\\\\|-)?({digit_pattern})*)"  # group1
measures_pattern = f"({'|'.join(different_measures)})"

# digit_pattern = "\d+((,|\.)?\d+)?"
# number_pattern = "({digit_pattern}(\/|\\|\\\\|-)?({digit_pattern})*|\d*\s*(½|¼|⅔|⅛|⅓))"  # group1
# measures_pattern = f"({'|'.join(different_measures)})"

In [244]:
def parse_number(str_num):
    if "-" in str_num:
        str_num = str_num.split("-")[-1]
    str_num = str_num.replace(",", ".").replace(" ", "")
    str_num = str_num.replace("\\", "/").replace("\\\\", "/")
    str_num = str_num.replace("½", "+0.5").replace("¼", "+0.25").replace("⅔", "+0.66").replace("⅛", "+0.13").replace("⅓", "+0.33")
    return eval(str_num)

In [245]:
re_pattern = re.compile(f"({number_pattern}\s*{measures_pattern}+\.?(?!\S)|{measures_pattern}\.?)")

In [246]:
re_pattern

re.compile(r'((\d*\s*(½|¼|⅔|⅛|⅓)|\d+((,|\.)?\d*)?(\/|\|\\|-)?(\d+((,|\.)?\d*)?)*)\s*(щеп(отка)?|ч\.\s?л|((ч(айн)?|ст)(олов)?\.?(ые|ых|ая|ой)?)?\s?ложе?к(а|и)?|гр?(амм)?|ст\.\s?л|зуб(чик)?\.?(а|ов)?|шт(ук(а|и)?)?|веточк(а|и)|кусоч\.?(ка|ек)?|кружк(а|и)?|ломтик|стеб(л(я|ей|евой)|ель(ка)?|)|чашк(а|и)?|банк(а|и)?|пуч(ок)?|килограмм|по( |\-)вкусу|лист(ы|ья|а|ьев|ов|ик(а|ов)?|очк(а|ов|и)?)|с?м|струч(ок|к(а|ов))|коч(ан)?|кг|м?л|головк(а|и)|ст(ак)?\.?(ан|ана)?)+\.?(?!\S)|(щеп(отка)?|ч\.\s?л|((ч(айн)?|ст)(олов)?\.?(ые|ых|ая|ой)?)?\s?ложе?к(а|и)?|гр?(амм)?|ст\.\s?л|зуб(чик)?\.?(а|ов)?|шт(ук(а|и)?)?|веточк(а|и)|кусоч\.?(ка|ек)?|кружк(а|и)?|ломтик|стеб(л(я|ей|евой)|ель(ка)?|)|чашк(а|и)?|банк(а|и)?|пуч(ок)?|килограмм|по( |\-)вкусу|лист(ы|ья|а|ьев|ов|ик(а|ов)?|очк(а|ов|и)?)|с?м|струч(ок|к(а|ов))|коч(ан)?|кг|м?л|головк(а|и)|ст(ак)?\.?(ан|ана)?)\.?)',
re.UNICODE)

In [247]:
from pymorphy2 import MorphAnalyzer
analyzer = MorphAnalyzer()

In [248]:
n_ingr = 0
n_success = 0
for recipe in downloaded_vegan_recipes:
    try:
        name, description, ingredients, script = process_recipe(recipe)
        processed_ingredients = []
        for ingr in ingredients:
    #         normalized = " ".join([analyzer.parse(w)[0].normal_form for w in ingr.split()])
            ingr = ingr.replace(";", " ").replace(":", " ").replace(")", " ").replace("(", " ").lower().strip()
            if ingr and ingr[-1] == ",":
                ingr = ingr[:-1]
            matches = re.finditer(re_pattern, ingr)
            ingr_cp = ingr
            split_word_indices = []
            prev_index = 0
            for w in ingr.split():
                ind = ingr_cp.index(w, prev_index)
                split_word_indices.append((ind-1, ind+len(w)))
                prev_index = ind+len(w)
            ingr_matches = []
            for m in matches:
                indices = m.span()
                for split_word_ind in split_word_indices:
                    if split_word_ind[1] > indices[1] and split_word_ind[0] < indices[0]:
                        break
                else:
                    ingr_matches.append(indices)
            if not len(ingr_matches):
                print(ingr, ingr_matches)
            else:
                parsed_ingr_matches = {}
                for ind in ingr_matches:
                    ingr_match = ingr[ind[0]:ind[1]]
                    number_ind = next(re.finditer(number_pattern, ingr_match)).span()
                    number = ingr_match[number_ind[0]:number_ind[1]].strip()
                    number = parse_number(number)
                    measure = ingr_match[:number_ind[0]] + ingr_match[number_ind[1]:].strip()
                    food = ingr[:ind[0]] + ingr[ind[1]:]
                    parsed_ingr_matches[food] = [number, measure]
                print(ingr, parsed_ingr_matches)
                n_success += 1
            n_ingr += 1
    except Exception as e:
        print(e)
#         print(recipe['text'])
print(f"{n_success/n_ingr:.2%} successful matches")

generator raised StopIteration
generator raised StopIteration
тесто []
вода — 100 мл {'вода — ': [100, 'мл']}
масло растительное — 100 мл {'масло растительное — ': [100, 'мл']}
соль — 0,5 ч. л. {'соль — ': [0.5, 'ч. л.']}
мука — 1,5-2 стак. {'мука — ': [2, 'стак.']}
начинка []
капуста квашеная — 300 г {'капуста квашеная — ': [300, 'г']}
лук репчатый — 1 шт. {'лук репчатый — ': [1, 'шт.']}
лук зеленый  перья  — 2-3 шт. {'лук зеленый  перья  — ': [3, 'шт.']}
• 400 мл сока/компота/морса {'•  сока/компота/морса': [400, 'мл']}
• 1 стакан черной смородины  клубника, голубика, черника {'•  черной смородины  клубника, голубика, черника': [1, 'стакан']}
• 1 стакан черешни/вишни без косточек {'•  черешни/вишни без косточек': [1, 'стакан']}
• 1 небольшое яблоко []
• 2 ч. л. агар-агара {'•  агар-агара': [2, 'ч. л.']}
рис — 125 г {'рис — ': [125, 'г']}
бульон  овощной или вода  — 400 мл {'бульон  овощной или вода  — ': [400, 'мл']}
кабачок  небольшой  — 1 шт {'кабачок  небольшой  — ': [1, 'шт']}
мо

на 14-17 штук {'на ': [17, 'штук']}
- 2 чашки тертой моркови {'-  тертой моркови': [2, 'чашки']}
- 1/2 чашки воды {'-  воды': [0.5, 'чашки']}
- 1/2 чайной ложки соли {'-  соли': [0.5, 'чайной ложки']}
- 3 чайные ложки сахара {'-  сахара': [3, 'чайные ложки']}
- 1/2 чашки манной крупы + для панировки {'-  манной крупы + для панировки': [0.5, 'чашки']}
- растительное масло для жарки []

• 1 репчатый лук []
• 2 зубчика чеснока {'•  чеснока': [2, 'зубчика']}
• 1 столовая ложка растительного масла {'•  растительного масла': [1, 'столовая ложка']}
• 2 небольшие моркови []
• 1 небольшой кабачок / цукини []
• 300гр риса {'•  риса': [300, 'гр']}
• 2 столовые ложки с горкой томатной пасты {'•  с горкой томатной пасты': [2, 'столовые ложки']}
• 450гр спелых томатов, или консервированных порубленных {'•  спелых томатов, или консервированных порубленных': [450, 'гр']}
• 2 ½ чашки  600мл  овощного бульона {'•   600мл  овощного бульона': [2.5, 'чашки'], '• 2 ½ чашки    овощного бульона': [600, 'мл']}

- капуста цветная — 700г {'- капуста цветная — ': [700, 'г']}
- помидор — 8-9шт {'- помидор — ': [9, 'шт']}
- лук репчатый — 3шт {'- лук репчатый — ': [3, 'шт']}
- чеснок — 2-3зуб. {'- чеснок — ': [3, 'зуб.']}
- масло растительное — 3-4 ст.л. {'- масло растительное — ': [4, 'ст.л.']}
- петрушка  укроп, базилик  — 1пуч. {'- петрушка  укроп, базилик  — ': [1, 'пуч.']}

- 80-100г растительного масла {'-  растительного масла': [100, 'г']}
- 2-3 большие головки лука []
- 2 большие моркови []
- 300-400г свежих шампиньонов {'-  свежих шампиньонов': [400, 'г']}
- 300г длиннозёрного риса {'-  длиннозёрного риса': [300, 'г']}
- 1,5 ч ложки соли, перец, специи. {'-  соли, перец, специи.': [1.5, 'ч ложки']}
- 250 г свежих шампиньонов {'-  свежих шампиньонов': [250, 'г']}
- 1 небольшая луковица []
- 1 небольшой красный перец []
- 2 средних помидора []
- 2 небольших зеленых перца []
- соль, молотый черный перец []
- банан — 6 шт. {'- банан — ': [6, 'шт.']}
- молоко 1% — 2/3 стак. {'- молоко 1% — ': 

- свежая или замороженная вишня — 450 г {'- свежая или замороженная вишня — ': [450, 'г']}
- мед — ¼ стак. {'- мед —': [0.25, 'стак.']}
- вода — 1 стак. {'- вода — ': [1, 'стак.']}
- 1 луковица []
- 2 моркови []
- 1 сладкий перец []
- 300 г шампиньонов {'-  шампиньонов': [300, 'г']}
- 150 г зеленого горошка {'-  зеленого горошка': [150, 'г']}
- 2-3 зубчика чеснока {'-  чеснока': [3, 'зубчика']}
- 1 ч.л. смеси прованских трав {'-  смеси прованских трав': [1, 'ч.л.']}

- бананы  средние  — 2 шт. {'- бананы  средние  — ': [2, 'шт.']}
- грецкий орех — 150 -200 г {'- грецкий орех — 150 -': [200, 'г']}
- миндаль — 150-200 г {'- миндаль — ': [200, 'г']}
- кокосовая стружка для обсыпки — 50 г. {'- кокосовая стружка для обсыпки — ': [50, 'г.']}
- яблоки 4-5 шт. {'- яблоки ': [5, 'шт.']}
- 1 банан []
- грецкие орехи 1 чашка  пиала  {'- грецкие орехи   пиала ': [1, 'чашка']}
- миндаль 1 чашка {'- миндаль ': [1, 'чашка']}
- семечки подсолнуха 1/3 чашки. {'- семечки подсолнуха ': [0.333333333333333

тыква – 1 кг {'тыква – ': [1, 'кг']}
нут консервированный/вареный – 500 г {'нут консервированный/вареный – ': [500, 'г']}
чеснок – 8 зубчиков {'чеснок – ': [8, 'зубчиков']}

- нут  350 гр {'- нут  ': [350, 'гр']}
- томатная паста  примерно 4 ст. л. с горкой {'- томатная паста  примерно  с горкой': [4, 'ст. л.']}

на 3 порции []
- помидоры  7 средних []
- чеснок  1 средняя головка []
- лук зелёный  1/2 пучка []
- петрушка  1/2 пучка []
- оливковое масло  1,5-2 ст. л. {'- оливковое масло  ': [2, 'ст. л.']}
специи []

- 2 ст. сухого гороха {'-  сухого гороха': [2, 'ст.']}
- 2 больших луковицы []
- соль, перец []
- зелень укропа []
- панировочные сухари или мука []
- растительное масло для жарки []
газированная вода – 180 мл {'газированная вода – ': [180, 'мл']}
мука – 350 г  не считая на подсыпку {'мука –   не считая на подсыпку': [350, 'г']}
сахар – 1 ст. л. {'сахар – ': [1, 'ст. л.']}
соль – 1 ч. л. {'соль – ': [1, 'ч. л.']}
сухие дрожжи – 1 ч. л. {'сухие дрожжи – ': [1, 'ч. л.']}
расти

...

1 головка чеснока {' чеснока': [1, 'головка']}
2 помидора []
2 ст. л. оливкового масла {' оливкового масла': [2, 'ст. л.']}
1 ч. л. паприки {' паприки': [1, 'ч. л.']}
2 ч. л. кориандра  цельного, не молотого! {' кориандра  цельного, не молотого!': [2, 'ч. л.']}
1/2 ч. л. сушёного тимьяна {' сушёного тимьяна': [0.5, 'ч. л.']}
2 стакана овощного бульона или воды {' овощного бульона или воды': [2, 'стакана']}
1 маленькая головка лука []

• 2 больших луковицы, нарезанных []
• 4 больших моркови, нарезанных []
• 6 зубчиков чеснока, пропущенных через пресс {'•  чеснока, пропущенных через пресс': [6, 'зубчиков']}
• 800гр томатов, консервированных или свежих {'•  томатов, консервированных или свежих': [800, 'гр']}
• 1 чашка красной чечевицы {'•  красной чечевицы': [1, 'чашка']}

250 г пасты ригатони  или другой {' пасты ригатони  или другой': [250, 'г']}
3-4 спелых помидора []
1,5 стак. готового нута  его можно купить уже готовый или отварить, предварительно замочив на ночь в воде {' готового н

баклажаны – 2 шт {'баклажаны – ': [2, 'шт']}
помидоры – 4 шт {'помидоры – ': [4, 'шт']}
фасоль отварная консервированная – 250 г {'фасоль отварная консервированная – ': [250, 'г']}
перец чили – 1/2 шт {'перец чили – ': [0.5, 'шт']}
лук – 100 г {'лук – ': [100, 'г']}
укроп – 10 г {'укроп – ': [10, 'г']}
кориандр молотый – 1 ч. л {'кориандр молотый – ': [1, 'ч. л']}
базилик – 3 веточки {'базилик – ': [3, 'веточки']}
чеснок – 10 г {'чеснок – ': [10, 'г']}

500 грамм шампиньонов {' шампиньонов': [500, 'грамм']}
2 стакана гречки {' гречки': [2, 'стакана']}
1 луковица []
оливковое масло []
1 ч. л. соли {' соли': [1, 'ч. л.']}
чечевица красная — 1 стакан {'чечевица красная — ': [1, 'стакан']}
вода — 5 стаканов []
масло сливочное топленое — 2 столовые ложки {'масло сливочное топленое — ': [2, 'столовые ложки']}
кумин  зира  — 1 чайная ложка {'кумин  зира  — ': [1, 'чайная ложка']}
семена горчицы черные — 1 чайная ложка {'семена горчицы черные — ': [1, 'чайная ложка']}
асафетида — 1 щепотка {'а

1. растительное молоко 300 г {'1. растительное молоко ': [300, 'г']}
2. растительное масло 40 г {'2. растительное масло ': [40, 'г']}
3. репчатый лук 2 шт. {'3. репчатый лук ': [2, 'шт.']}
4. чеснок 2 зубца []
5. цукини 1 шт. {'5. цукини ': [1, 'шт.']}
6. баклажан 1 шт. {'6. баклажан ': [1, 'шт.']}
7. картофель 2 шт. {'7. картофель ': [2, 'шт.']}
8. морковь 2 шт. {'8. морковь ': [2, 'шт.']}
9. помидор 3 шт. {'9. помидор ': [3, 'шт.']}
10. нут варенный 400 г {'10. нут варенный ': [400, 'г']}
11. шампиньоны 200 г {'11. шампиньоны ': [200, 'г']}
12. куркума 1 ч. л. {'12. куркума ': [1, 'ч. л.']}
13. кумин 1 ч. л. {'13. кумин ': [1, 'ч. л.']}
14. семена горчицы 1 ч. л. {'14. семена горчицы ': [1, 'ч. л.']}
15. листья карри 8 шт. {' карри 8 шт.': [15.0, 'листья'], '15. листья карри ': [8, 'шт.']}
16. красный чили 1 шт. {'16. красный чили ': [1, 'шт.']}
17. рис басмати 200 г {'17. рис басмати ': [200, 'г']}
цуккини 1 []
морковка 1 []
помидорка 1 []
грибочки 6 []
ростки сои/пшеницы []
зелень 

кориандр — 0,5 ч. л. {'кориандр — ': [0.5, 'ч. л.']}
порошок карри — 1 ч. л. {'порошок карри — ': [1, 'ч. л.']}
сушеный красный перец – 1 шт. {'сушеный красный перец – ': [1, 'шт.']}
соль — 0,5 ч. л. {'соль — ': [0.5, 'ч. л.']}

• 150г кураги  мягкой {'•  кураги  мягкой': [150, 'г']}
• 100г сушеного ананаса  или другой фрукт по желанию {'•  сушеного ананаса  или другой фрукт по желанию': [100, 'г']}
• 3/4 стакана поджаренного миндаля {'•  поджаренного миндаля': [0.75, 'стакана']}
• 1 стакан  220г  овсяных хлопьев {'•   220г  овсяных хлопьев': [1, 'стакан'], '• 1 стакан    овсяных хлопьев': [220, 'г']}
• 100-150мл апельсинового сока {'•  апельсинового сока': [150, 'мл']}
- 1 стакан гречишной муки {'-  гречишной муки': [1, 'стакан']}
- 1 стакан геркулеса {'-  геркулеса': [1, 'стакан']}
- 1 среднее яблоко, натереть []
- 1 большой спелый банан, пюрировать []
- 6 фиников без косточки, нарезать []
- 1 чайная ложка корицы {'-  корицы': [1, 'чайная ложка']}
дополнительно []
- 1 столовая ложка 

- 6 ст.л. закваски для опары  http //vk.com/vegarecipes?w=wall-35838585_11655 {'-  закваски для опары  http //vk.com/vegarecipes?w=wall-35838585_11655': [6, 'ст.л.']}
- 1 стакан ржаной муки {'-  ржаной муки': [1, 'стакан']}
- 1,5 стакана пшеничной муки  из них 0,5 стакана для опары {'-  пшеничной муки  из них 0,5 стакана для опары': [1.5, 'стакана'], '- 1,5 стакана пшеничной муки  из них  для опары': [0.5, 'стакана']}
- 4 ст.л. солода {'-  солода': [4, 'ст.л.']}
- 1 стакан воды, молока или сыворотки  подойдет все что есть в доме {'-  воды, молока или сыворотки  подойдет все что есть в доме': [1, 'стакан']}
- 2 ст.л. подсолнечного масла {'-  подсолнечного масла': [2, 'ст.л.']}
- 2 ст.л. сахара {'-  сахара': [2, 'ст.л.']}
- 1 ч. л. соли {'-  соли': [1, 'ч. л.']}
- 1-2 ч.л. кориандра {'-  кориандра': [2, 'ч.л.']}
- около 300 гр муки {'- около  муки': [300, 'гр']}
- 300 мл. воды {'-  воды': [300, 'мл.']}

5 ст.л. ягод облепихи  свежих или замороженных {' ягод облепихи  свежих или заморожен

In [186]:
ingr.split()

['-', 'мука', '-', 'до', 'густоты', 'сметаны']

In [19]:
ingr

'Рис — 125 г'